# Libraries

In [23]:
import pandas as pd
import numpy as np
import math
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import preprocessing

# Dataset Pre-Processing


In [10]:
# TODO : Copy Dataframes here
with open('travel_cb.txt') as f:
    lines = f.readlines()

for i in range(len(lines)):
    lines[i]=lines[i].strip()
    
count = 0
dic = {}
inner = {}
for i in range(len(lines)):
    dic[count] = {}
    pos = lines[i].find(":")
    if(pos != -1):
        attr = lines[i][:pos]
        val = lines[i][pos+2:len(lines[i])-1].strip()
        inner[attr] = val
        if(attr == "Hotel"):
            dic[count] = inner
            inner = {}
            count += 1
print(dic[10])

{'JourneyCode': '11', 'HolidayType': 'City', 'Price': '1978', 'NumberOfPersons': '2', 'Region': 'Cairo', 'Transportation': 'Plane', 'Duration': '7', 'Season': 'April', 'Accommodation': 'ThreeStars', 'Hotel': '"Hotel Victoria, Cairo"'}


In [49]:
main=pd.DataFrame.from_dict(dic,orient="index")
N = 3
# Drop last N columns of dataframe
df = main.iloc[: , :-N]
# drop rows with NaN as values
df = df.dropna()
# add feature - weight
df['Weight'] = 1
df

,JourneyCode,HolidayType,Price,NumberOfPersons,Region,Transportation,Duration,Season,Accommodation,Hotel,Weight
0,1,Bathing,2498,2,Egypt,Plane,14,April,TwoStars,"""Hotel White House, Egypt""",1
1,2,Bathing,3066,3,Egypt,Plane,21,May,TwoStars,"""Hotel White House, Egypt""",1
2,3,Active,2778,2,Egypt,Plane,14,June,TwoStars,"""Hotel Geisum, Egypt""",1
3,4,Bathing,3195,3,Egypt,Plane,7,July,ThreeStars,"""Hotel Al Mashrabia, Egypt""",1
4,5,Bathing,3158,2,Egypt,Plane,14,September,ThreeStars,"""Hotel Al Mashrabia, Egypt""",1
...,...,...,...,...,...,...,...,...,...,...,...
1471,1443,Bathing,1518,2,TurkishRiviera,Plane,14,December,ThreeStars,"""Hotel Santana, Turk. Riviera""",1
1473,1465,Recreation,1878,2,Tunisia,Plane,14,November,ThreeStars,"""Hotel Quatre Saisons, Tunisia""",1
1475,1466,Recreation,1765,3,Tunisia,Plane,7,January,ThreeStars,"""Hotel Quatre Saisons, Tunisia""",1
1477,1465,Recreation,1878,2,Tunisia,Plane,14,November,ThreeStars,"""Hotel Quatre Saisons, Tunisia""",1


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1475 entries, 0 to 1479
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   JourneyCode      1475 non-null   object
 1   HolidayType      1475 non-null   object
 2   Price            1475 non-null   object
 3   NumberOfPersons  1475 non-null   object
 4   Region           1475 non-null   object
 5   Transportation   1475 non-null   object
 6   Duration         1475 non-null   object
 7   Season           1475 non-null   object
 8   Accommodation    1475 non-null   object
 9   Hotel            1475 non-null   object
 10  Weight           1475 non-null   int64 
dtypes: int64(1), object(10)
memory usage: 138.3+ KB


In [51]:
# TODO: Normalize data into numerical values


#creating labelEncoder
le = preprocessing.LabelEncoder()
# Converting string labels into numbers.
df['Transportation']=le.fit_transform(df['Transportation'])
df['Season']=le.fit_transform(df['Season'])
df['HolidayType']=le.fit_transform(df['HolidayType'])
df['Region']=le.fit_transform(df['Region'])
df['Accommodation']=le.fit_transform(df['Accommodation'])
df['Hotel']=le.fit_transform(df['Hotel'])
df.head()

# df.Transportation.unique()

,JourneyCode,HolidayType,Price,NumberOfPersons,Region,Transportation,Duration,Season,Accommodation,Hotel,Weight
0,1,1,2498,2,27,2,14,0,5,413,1
1,2,1,3066,3,27,2,21,8,5,413,1
2,3,0,2778,2,27,2,14,6,5,260,1
3,4,1,3195,3,27,2,7,5,4,187,1
4,5,1,3158,2,27,2,14,11,4,187,1


## Data Split

In [58]:
# TODO: split data as train and test
X = df.iloc[:,1:]
X = X.drop(df.columns[[2]], axis=1)  # df.columns is zero-based pd.Index
#X.head()
X = X.values
X

array([[1, '2', 27, ..., 5, 413, 1],
       [1, '3', 27, ..., 5, 413, 1],
       [0, '2', 27, ..., 5, 260, 1],
       ...,
       [5, '3', 60, ..., 4, 354, 1],
       [5, '2', 60, ..., 4, 354, 1],
       [5, '3', 60, ..., 4, 354, 1]], dtype=object)

# KNN Algorithm

In [15]:
def euclidean_distance(self,a,b,w = 1): # initial weight = 1
    return np.sqrt(np.sum(w*((a-b)**2)))

class KNN:
    def __init__(self, K = 3):
        self.K = K
    
    def fit(self,X,y):
        self.X_train = X
        self.y_train = y
        
    def predict(self,samples):
        predicted_labels = [self._predict(sample) for sample in samples]
        return np.array(predicted_labels)
    
    def _predict(self,sample):
        # distance between sample and training points
        distances = [euclidean_distance(sample,self.x_train) for x_train in self.X_train]
        
        # get k-nearest data after sorting
        k_nearest_indices = np.sort(distances)[:self.K]
        # get k-nearest data labels
        k_nearest_labels = [self.y_train[i] for i in k_nearest_indices]
        # arrange frequency to get common class label
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0] 

In [ ]:
# splitting data - 0.1%
# user query - given
# three best choices - 
# generating 3 hard coded questions - ask about questions.